In [1]:
from dotenv import load_dotenv
import os

# Cargamos el archivo .env
load_dotenv('../.env')

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

## Cargar documentos y generar chunks

In [3]:
from pathlib import Path
from langchain_text_splitters import MarkdownHeaderTextSplitter
from typing import List, Dict, Any

def build_enriched_content(content: str, metadata: dict, document_name: str) -> str:
    """
    Construye el contenido enriquecido agregando documento y headers al inicio
    
    Args:
        content: Contenido original del chunk
        metadata: Metadatos con información de headers
        document_name: Nombre del documento (sin extensión)
    
    Returns:
        Contenido enriquecido con documento y headers
    """
    header_parts = []
    
    # Agregar nombre del documento al inicio
    header_parts.append(f"[Documento: {document_name}]")
    
    # Extraer headers en orden jerárquico
    for i in range(1, 6):  # Header 1 a Header 5
        header_key = f"Header {i}"
        if header_key in metadata:
            if i == 1:
                header_parts.append(f"[Titulo: {metadata[header_key]}]")
            else:
                header_parts.append(f"[Subtitulo: {metadata[header_key]}]")
    
    # Combinar documento + headers + contenido
    enriched_content = "".join(header_parts) + " " + content
    
    return enriched_content

In [4]:
def load_and_chunk_markdown_with_full_context(data_dir_relative: str = "../data") -> List[Dict[str, Any]]:
    """
    Carga documentos markdown y crea chunks enriquecidos con contexto completo
    (documento + headers)
    
    Args:
        data_dir_relative: Ruta relativa a la carpeta de datos
    
    Returns:
        Lista de diccionarios con chunks enriquecidos y metadatos
    """
    
    # Definir la ruta a la carpeta data
    data_dir = Path.cwd().parent / "data" if data_dir_relative == "../data" else Path(data_dir_relative)
    
    if not data_dir.exists():
        print(f"La carpeta {data_dir} no existe")
        return []
    
    # Configurar headers para dividir
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"), 
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
    ]
    
    # Crear el splitter de markdown
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on,
        #return_each_line=True,
    )
    
    all_chunks = []
    processed_files = 0
    
    # Procesar todos los archivos markdown
    for md_file in data_dir.glob('*.md'):
        try:
            # Leer el contenido del archivo
            with open(md_file, 'r', encoding='utf-8') as file:
                markdown_document = file.read()
            
            # Obtener nombre del documento sin extensión
            document_name = md_file.stem
            
            # Dividir el documento
            md_header_splits = markdown_splitter.split_text(markdown_document)
            
            # Procesar cada chunk
            for i, document in enumerate(md_header_splits):
                # Contenido original
                original_content = document.page_content
                
                # Construir contenido enriquecido con documento + headers
                enriched_content = build_enriched_content(
                    original_content, 
                    document.metadata, 
                    document_name
                )
                
                chunk_data = {
                    'content': enriched_content,  # Contenido enriquecido para la BD vectorial
                    'original_content': original_content,  # Contenido original
                    'metadata': {
                        **document.metadata,  # Headers metadata
                        'source_file': md_file.name,
                        'source_path': str(md_file),
                        'document_name': document_name,
                        'chunk_index': i,
                        'total_chunks_in_file': len(md_header_splits),
                        'has_headers': bool([k for k in document.metadata.keys() if 'Header' in k])
                    }
                }
                all_chunks.append(chunk_data)
            
            processed_files += 1
            print(f"Procesado: {md_file.name} ({document_name}) - {len(md_header_splits)} chunks")
            print("-" * 100)
            print(md_header_splits) 
            print("-" * 100)
            
        except Exception as e:
            print(f"Error procesando {md_file}: {e}")
    
    print(f"\n Procesados {processed_files} archivos")
    print(f" Total de chunks generados: {len(all_chunks)}")
    
    return all_chunks

In [5]:
# Cargar documentos con contexto completo
chunks = load_and_chunk_markdown_with_full_context()

Procesado: programas.md (programas) - 1 chunks
----------------------------------------------------------------------------------------------------
[Document(metadata={'Header 1': 'Programas', 'Header 2': 'Lista de programas'}, page_content='Los programas que se tienen son:  \n- Data Engineer: El programa de Data Engineer está diseñado para formar profesionales capaces de diseñar, construir y mantener sistemas de datos a gran escala. Los estudiantes aprenderán a trabajar con tecnologías modernas de big data, pipelines de datos y arquitecturas distribuidas.\n- Data Architect: El programa de Data Architect forma especialistas en el diseño de arquitecturas de datos empresariales. Los participantes desarrollarán habilidades para crear estrategias de datos, gobiernos de información y arquitecturas técnicas escalables.\n- ML Engineer: El programa de ML Engineer prepara profesionales para implementar y mantener sistemas de machine learning en producción. Los estudiantes aprenderán desde el de

In [6]:
# Mostrar ejemplos de los chunks enriquecidos
print("\n" + "="*80)
print("EJEMPLOS DE CHUNKS ENRIQUECIDOS CON DOCUMENTO:")
print("="*80)

for i, chunk in enumerate(chunks[:5]):
    print(f"\n--- CHUNK {i+1} ---")
    print(f"Archivo: {chunk['metadata']['source_file']}")
    print(f"Documento: {chunk['metadata']['document_name']}")
    
    # Mostrar contenido enriquecido (para BD vectorial)
    print(f"\n CONTENIDO ENRIQUECIDO COMPLETO:")
    print(f"'{chunk['content']}'")
    
    # Mostrar contenido original
    print(f"\n CONTENIDO ORIGINAL:")
    print(f"'{chunk['original_content']}'")
    
    print("-" * 60)


EJEMPLOS DE CHUNKS ENRIQUECIDOS CON DOCUMENTO:

--- CHUNK 1 ---
Archivo: programas.md
Documento: programas

 CONTENIDO ENRIQUECIDO COMPLETO:
'[Documento: programas][Titulo: Programas][Subtitulo: Lista de programas] Los programas que se tienen son:  
- Data Engineer: El programa de Data Engineer está diseñado para formar profesionales capaces de diseñar, construir y mantener sistemas de datos a gran escala. Los estudiantes aprenderán a trabajar con tecnologías modernas de big data, pipelines de datos y arquitecturas distribuidas.
- Data Architect: El programa de Data Architect forma especialistas en el diseño de arquitecturas de datos empresariales. Los participantes desarrollarán habilidades para crear estrategias de datos, gobiernos de información y arquitecturas técnicas escalables.
- ML Engineer: El programa de ML Engineer prepara profesionales para implementar y mantener sistemas de machine learning en producción. Los estudiantes aprenderán desde el desarrollo de modelos hasta su 

# Base de datos vectorial

## Crear coleccion

In [7]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.exceptions import UnexpectedResponse

# Verificar variables de entorno
qdrant_url = os.getenv('QDRANT_URL')
qdrant_key = os.getenv('QDRANT_KEY')
collection_name = os.getenv('QDRANT_COLLECTION_NAME')

# Validar que las variables estén configuradas
if not all([qdrant_url, qdrant_key, collection_name]):
    missing = []
    if not qdrant_url: missing.append('QDRANT_URL')
    if not qdrant_key: missing.append('QDRANT_KEY')
    if not collection_name: missing.append('QDRANT_COLLECTION_NAME')
    raise ValueError(f"Variables de entorno faltantes: {', '.join(missing)}")

print(f"Conectando a Qdrant: {qdrant_url}")
print(f"Colección objetivo: {collection_name}")

try:
    # Crear cliente
    client = QdrantClient(
        url=qdrant_url,
        api_key=qdrant_key,
        timeout=60
    )
    
    # Verificar conexión
    print("Verificando conexión a Qdrant...")
    collections = client.get_collections()
    print(f"Conexión exitosa. Colecciones existentes: {len(collections.collections)}")
    
    # Verificar si la colección existe usando el método recomendado
    if client.collection_exists(collection_name):
        print(f"La colección {collection_name} existe, eliminándola...")
        try:
            client.delete_collection(collection_name=collection_name)
            print(f"Colección {collection_name} eliminada correctamente")
        except UnexpectedResponse as e:
            print(f"Error al eliminar colección: {e}")
    else:
        print(f"La colección {collection_name} no existe, procediendo a crearla")

    print(f"Creando la colección {collection_name}")

    # Crear colección usando el método recomendado
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=1536,  # dimensión del embedding
            distance=Distance.COSINE  # COSINE ideal para embeddings normalizados
        )
    )

    print(f"Colección {collection_name} creada correctamente.")
  
except Exception as e:
    print(f"Error inesperado: {type(e).__name__}: {e}")

/home/igorov/dev/miniconda3/envs/v310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Conectando a Qdrant: https://5d3a3746-9bf9-4feb-8c37-3e0fecba6e25.us-east4-0.gcp.cloud.qdrant.io
Colección objetivo: techFlow-academy
Verificando conexión a Qdrant...
Conexión exitosa. Colecciones existentes: 3
La colección techFlow-academy existe, eliminándola...
Colección techFlow-academy eliminada correctamente
Creando la colección techFlow-academy
Colección techFlow-academy creada correctamente.


## Subir embeddings

In [8]:
# Modelo de embeddings
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

In [9]:
from langchain_core.documents import Document
from typing import List, Dict, Any

def convert_chunks_to_documents(chunks: List[Dict[str, Any]]) -> List[Document]:
    documents = []
    
    for chunk in chunks:
        doc = Document(
            page_content=chunk['content'],
            metadata=chunk['metadata']
        )
        documents.append(doc)
    
    return documents

In [10]:
# Convertir chunks a Documents de LangChain
docs = convert_chunks_to_documents(chunks)

In [11]:
from langchain_qdrant import QdrantVectorStore

# Crear el vector store en Qdrant
qdrant = QdrantVectorStore.from_documents(
    docs,
    embeddings_model,
    url=qdrant_url,
    prefer_grpc=True,
    api_key=qdrant_key,
    collection_name=collection_name,
)